In [21]:
import pandas as pd
import numpy as np

In [124]:
SEASON = 9

# to be done: make this scalable to defenses too (no easy way to verify data)
all = pd.read_csv(f'../s{SEASON}/s{SEASON}.csv')
atk_df = all[all['type'] == 'atk']

# also
striker_df = pd.read_csv('apply_figs/f_striker_attributes.csv')
special_df = pd.read_csv('apply_figs/f_special_attributes.csv')

# and
affinity_map = pd.read_csv('save_figs/terrain.csv')
color_map = pd.read_csv('save_figs/color_map.csv')

# using these dfs
atk_df
striker_df
special_df
affinity_map
color_map

# properties
list_str = ['d1','d2','d3','d4','a1','a2','a3','a4']; list_sp = ['ds1','ds2','as1','as2']

# could consider splits across striker + specials
def_slots = ['d1', 'd2', 'd3', 'd4', 'ds1', 'ds2']
atk_slots = ['a1', 'a2', 'a3', 'a4', 'as1', 'as2']

def_slots_strikers = ['d1', 'd2', 'd3', 'd4']
atk_slots_strikers = ['a1', 'a2', 'a3', 'a4']

# partial info
def_slots_partial = ['d1','ds1','ds2']


In [99]:
pd.set_option('display.max_columns', None)

In [100]:
# expands original dataset to include char-specific features, defined in respective dfs
def expand_features_wide(df: pd.DataFrame, list_ftrs_to_expand: list[str], ftr_to_merge: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    for slot in list_ftrs_to_expand:
        tmp = ftr_to_merge.copy()
        tmp.columns = [f'{slot}_{c}' for c in tmp.columns]
        out = out.merge(tmp, how='left', left_on=slot, right_on=f'{slot}_char', validate='m:1')
        out = out.drop(columns=[f'{slot}_char'])
    
    return out

In [101]:
terrain_drops = {
    "urban": ("_outdoor", "_indoor"),
    "outdoor": ("_urban", "_indoor"),
    "indoor": ("_urban", "_outdoor")}

# removes extraneous terrain attributes on map affinity

def remove_terrains(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    terrain = df['affinity'].iloc[0]
    
    to_drop = terrain_drops.get(terrain)
    cols_dropped = [c for c in out.columns if c.endswith(to_drop)]
    out = out.drop(columns=cols_dropped)

    return out

In [148]:
# convert terrain multipliers into features

def expand_terrain_cols(df: pd.DataFrame, affinity_map: pd.DataFrame) -> pd.DataFrame:

    out = df.copy()
    suffixes = ('dmg_mult', 'block_r')
    terrain = atk_df["affinity"].iloc[0]

    aff_cols = [c for c in out.columns if c.endswith(f'_affinity_{terrain}')]

    maps = {s: dict(zip(affinity_map['affinity'], affinity_map[s])) for s in suffixes}

    for col in aff_cols:
        for s in suffixes:
            out[f'{col}_{s}'] = out[col].map(maps[s])

    out = out.drop(columns=aff_cols)
    
    return out

In [137]:
# add some team classifiers

# add up d1-d4 roles, make a new column, same for a1-a4 roles # make one for each unique role, so two extra sets of columns with role number of columns there
# do the same for position
# aggregate sum hp, mean hp, min hp, max hp, same for atk, same for def, then also do the subtraction

# adds row-wise aggregates for a given stat
def add_team_aggregates_cont(
    df: pd.DataFrame, stat: str, def_slots_f: list[str], def_slots_p: list[str], atk_slots: list[str]) -> pd.DataFrame:
    
    out = df.copy()

    def_cols_f = [f"{s}_{stat}" for s in def_slots_f]
    def_cols_p = [f"{s}_{stat}" for s in def_slots_p]
    atk_cols = [f"{s}_{stat}" for s in atk_slots]

    # aggregate row-wise
    # all units (post)
    out[f"def_{stat}_sum_FULL"]  = out[def_cols_f].sum(axis=1)
    out[f"def_{stat}_mean_FULL"] = out[def_cols_f].mean(axis=1)
    out[f"def_{stat}_std_FULL"] = out[def_cols_f].std(axis=1)
    out[f"def_{stat}_min_FULL"]  = out[def_cols_f].min(axis=1)
    out[f"def_{stat}_max_FULL"]  = out[def_cols_f].max(axis=1)

    # seen units (pre)
    out[f"def_{stat}_sum_PARTIAL"]  = out[def_cols_p].sum(axis=1)
    out[f"def_{stat}_mean_PARTIAL"] = out[def_cols_p].mean(axis=1)
    out[f"def_{stat}_std_PARTIAL"] = out[def_cols_p].std(axis=1)
    out[f"def_{stat}_min_PARTIAL"]  = out[def_cols_p].min(axis=1)
    out[f"def_{stat}_max_PARTIAL"]  = out[def_cols_p].max(axis=1)

    # always have own units
    out[f"atk_{stat}_sum"]  = out[atk_cols].sum(axis=1)
    out[f"atk_{stat}_mean"] = out[atk_cols].mean(axis=1)
    out[f"atk_{stat}_std"] = out[atk_cols].std(axis=1)
    out[f"atk_{stat}_min"]  = out[atk_cols].min(axis=1)
    out[f"atk_{stat}_max"]  = out[atk_cols].max(axis=1)

    # diffs between atk and def teams
    out[f"diff_{stat}_sum_PARTIAL"] = out[f"atk_{stat}_sum"] - out[f"def_{stat}_sum_PARTIAL"]
    out[f"diff_{stat}_mean_PARTIAL"] = out[f"atk_{stat}_mean"] - out[f"def_{stat}_mean_PARTIAL"]
    out[f"diff_{stat}_std_PARTIAL"] = out[f"atk_{stat}_std"] - out[f"def_{stat}_std_PARTIAL"]

    out[f"diff_{stat}_sum_FULL"] = out[f"atk_{stat}_sum"] - out[f"def_{stat}_sum_FULL"]
    out[f"diff_{stat}_mean_FULL"] = out[f"atk_{stat}_mean"] - out[f"def_{stat}_mean_FULL"]
    out[f"diff_{stat}_std_FULL"] = out[f"atk_{stat}_std"] - out[f"def_{stat}_std_FULL"]

    return out

In [142]:
# adds row-wise aggregates for a given categorical stat
def add_team_aggregates_cat(
    df: pd.DataFrame, stat: str, def_slots_f: list[str], atk_slots: list[str]) -> pd.DataFrame:

    out = df.copy()

    def_cols_f = [f"{s}_{stat}" for s in def_slots_f]
    atk_cols = [f"{s}_{stat}" for s in atk_slots]

    def_stacked = out[def_cols_f].stack()
    atk_stacked = out[atk_cols].stack()

    def_counts = pd.get_dummies(def_stacked).groupby(level=0).sum().reindex(out.index)
    atk_counts = pd.get_dummies(atk_stacked).groupby(level=0).sum().reindex(out.index)

    def_props = def_counts/len(def_cols_f)
    atk_props = atk_counts/len(atk_cols)

    def_props.columns = [f"def_{stat}_{c}_prop_FULL" for c in def_props.columns]
    atk_props.columns = [f"atk_{stat}_{c}_prop_FULL" for c in atk_props.columns]

    out = out.join(def_props).join(atk_props)

    return out

In [149]:
# pipeline

copy = atk_df.copy()

c = expand_features_wide(copy, list_str, striker_df)

c = expand_features_wide(c, list_sp, special_df)

c = remove_terrains(c)

c = expand_terrain_cols(c, affinity_map)

# these can change, speculating that these are most imp
for stat_cont in ['ex_cost', 'ex_length', 'max_hp', 'atk', 'def', 'healing', 'accuracy', 'evasion', 'crit']:
    c = add_team_aggregates_cont(c, stat_cont, def_slots, def_slots_partial, atk_slots)

# total categorical vars, also can add more, in terms of strikers-only, encode roles and positioning info (heuristic)
for stat_cat in ['role', 'position']:
    c = add_team_aggregates_cat(c, stat_cat, def_slots_strikers, atk_slots_strikers)

c


,affinity,type,d1,d2,d3,d4,ds1,ds2,a1,a2,a3,a4,as1,as2,outcome,rank,d1_atk_type,d1_def_type,d1_role,d1_weapon,d1_position,d1_alt_count,d1_invul,d1_deals_dmg,d1_heals,d1_heals_team_or_self,d1_ex_cost,d1_ex_length,d1_generates_cost,d1_taunts,d1_max_hp,d1_atk,d1_def,d1_dmg_reduction,d1_healing,d1_accuracy,d1_evasion,d1_crit,d1_crit_res,d1_crit_dmg,d1_crit_dmg_res,d1_stability,d1_min_dmg,d1_n_range,d1_cc_power,d1_cc_res,d1_recov_boost,d1_atk_spd,d1_mov_spd,d1_block_rate_bonus,d1_def_piercing,d1_mag_count,d1_na_cost,d1_equip_slot1,d1_equip_slot2,d1_equip_slot3,d2_atk_type,d2_def_type,d2_role,d2_weapon,d2_position,d2_alt_count,d2_invul,d2_deals_dmg,d2_heals,d2_heals_team_or_self,d2_ex_cost,d2_ex_length,d2_generates_cost,d2_taunts,d2_max_hp,d2_atk,d2_def,d2_dmg_reduction,d2_healing,d2_accuracy,d2_evasion,d2_crit,d2_crit_res,d2_crit_dmg,d2_crit_dmg_res,d2_stability,d2_min_dmg,d2_n_range,d2_cc_power,d2_cc_res,d2_recov_boost,d2_atk_spd,d2_mov_spd,d2_block_rate_bonus,d2_def_piercing,d2_mag_count,d2_na_cost,d2_equip_slot1,d2_equip_slot2,d2_equip_slot3,d3_atk_type,d3_def_type,d3_role,d3_weapon,d3_position,d3_alt_count,d3_invul,d3_deals_dmg,d3_heals,d3_heals_team_or_self,d3_ex_cost,d3_ex_length,d3_generates_cost,d3_taunts,d3_max_hp,d3_atk,d3_def,d3_dmg_reduction,d3_healing,d3_accuracy,d3_evasion,d3_crit,d3_crit_res,d3_crit_dmg,d3_crit_dmg_res,d3_stability,d3_min_dmg,d3_n_range,d3_cc_power,d3_cc_res,d3_recov_boost,d3_atk_spd,d3_mov_spd,d3_block_rate_bonus,d3_def_piercing,d3_mag_count,d3_na_cost,d3_equip_slot1,d3_equip_slot2,d3_equip_slot3,d4_atk_type,d4_def_type,d4_role,d4_weapon,d4_position,d4_alt_count,d4_invul,d4_deals_dmg,d4_heals,d4_heals_team_or_self,d4_ex_cost,d4_ex_length,d4_generates_cost,d4_taunts,d4_max_hp,d4_atk,d4_def,d4_dmg_reduction,d4_healing,d4_accuracy,d4_evasion,d4_crit,d4_crit_res,d4_crit_dmg,d4_crit_dmg_res,d4_stability,d4_min_dmg,d4_n_range,d4_cc_power,d4_cc_res,d4_recov_boost,d4_atk_spd,d4_mov_spd,d4_block_rate_bonus,d4_def_piercing,d4_mag_count,d4_na_cost,d4_equip_slot1,d4_equip_slot2,d4_equip_slot3,a1_atk_type,a1_def_type,a1_role,a1_weapon,a1_position,a1_alt_count,a1_invul,a1_deals_dmg,a1_heals,a1_heals_team_or_self,a1_ex_cost,a1_ex_length,a1_generates_cost,a1_taunts,a1_max_hp,a1_atk,a1_def,a1_dmg_reduction,a1_healing,a1_accuracy,a1_evasion,a1_crit,a1_crit_res,a1_crit_dmg,a1_crit_dmg_res,a1_stability,a1_min_dmg,a1_n_range,a1_cc_power,a1_cc_res,a1_recov_boost,a1_atk_spd,a1_mov_spd,a1_block_rate_bonus,a1_def_piercing,a1_mag_count,a1_na_cost,a1_equip_slot1,a1_equip_slot2,a1_equip_slot3,a2_atk_type,a2_def_type,a2_role,a2_weapon,a2_position,a2_alt_count,a2_invul,a2_deals_dmg,a2_heals,a2_heals_team_or_self,a2_ex_cost,a2_ex_length,a2_generates_cost,a2_taunts,a2_max_hp,a2_atk,a2_def,a2_dmg_reduction,a2_healing,a2_accuracy,a2_evasion,a2_crit,a2_crit_res,a2_crit_dmg,a2_crit_dmg_res,a2_stability,a2_min_dmg,a2_n_range,a2_cc_power,a2_cc_res,a2_recov_boost,a2_atk_spd,a2_mov_spd,a2_block_rate_bonus,a2_def_piercing,a2_mag_count,a2_na_cost,a2_equip_slot1,a2_equip_slot2,a2_equip_slot3,a3_atk_type,a3_def_type,a3_role,a3_weapon,a3_position,a3_alt_count,a3_invul,a3_deals_dmg,a3_heals,a3_heals_team_or_self,a3_ex_cost,a3_ex_length,a3_generates_cost,a3_taunts,a3_max_hp,a3_atk,a3_def,a3_dmg_reduction,a3_healing,a3_accuracy,a3_evasion,a3_crit,a3_crit_res,a3_crit_dmg,a3_crit_dmg_res,a3_stability,a3_min_dmg,a3_n_range,a3_cc_power,a3_cc_res,a3_recov_boost,a3_atk_spd,a3_mov_spd,a3_block_rate_bonus,a3_def_piercing,a3_mag_count,a3_na_cost,a3_equip_slot1,a3_equip_slot2,a3_equip_slot3,a4_atk_type,a4_def_type,a4_role,a4_weapon,a4_position,a4_alt_count,a4_invul,a4_deals_dmg,a4_heals,a4_heals_team_or_self,a4_ex_cost,a4_ex_length,a4_generates_cost,a4_taunts,a4_max_hp,a4_atk,a4_def,a4_dmg_reduction,a4_healing,a4_accuracy,a4_evasion,a4_crit,a4_crit_res,a4_crit_dmg,a4_crit_dmg_res,a4_stability,a4_min_dmg,a4_n_range,a4_cc_power,a4_cc_res,a4_recov_boost,a4_atk_spd,a4_mov_spd,a4_block_rate_bonus,a4_def_piercing,a4_mag_count,a4_na_cost,a4_equip_slot1,a4_equip_s